In [76]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
!pip list

In [78]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

In [79]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [80]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [81]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [82]:
DATA_PATH = os.path.join('data1') 

sign = np.array(['dvsign', 'other'])

no_sequences = 90

sequence_length = 40

In [83]:
for action in sign: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in sign:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, '{}. Collecting frames for {} Video Number {}'.format(frame_num, action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    cap.release()
                    cv2.destroyAllWindows()
                    break
                    

In [74]:
cap.release()
cv2.destroyAllWindows()

In [84]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [85]:
label_map = {label:num for num, label in enumerate(sign)}

In [86]:
label_map

{'dvsign': 0, 'other': 1}

In [87]:
sequences, labels = [], []
for action in sign:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [88]:
np.array(sequences).shape

(180, 40, 126)

In [89]:
np.array(labels).shape

(180,)

In [90]:
X = np.array(sequences)

In [91]:
X.shape

(180, 40, 126)

In [92]:
y = to_categorical(labels).astype(int)

In [93]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1,

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 42)

In [95]:
y_test.shape

(18, 2)

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [107]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(40,126)))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(sign.shape[0], activation='softmax'))

In [108]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [109]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
6/6 [==============================] - 5s 44ms/step - loss: 0.6896 - categorical_accuracy: 0.4877
Epoch 2/200
6/6 [==============================] - 0s 49ms/step - loss: 0.6725 - categorical_accuracy: 0.6852
Epoch 3/200
6/6 [==============================] - 0s 47ms/step - loss: 0.6589 - categorical_accuracy: 0.6914
Epoch 4/200
6/6 [==============================] - 0s 40ms/step - loss: 0.6258 - categorical_accuracy: 0.6914
Epoch 5/200
6/6 [==============================] - 0s 41ms/step - loss: 0.6088 - categorical_accuracy: 0.7346
Epoch 6/200
6/6 [==============================] - 0s 45ms/step - loss: 0.9499 - categorical_accuracy: 0.7469
Epoch 7/200
6/6 [==============================] - 0s 41ms/step - loss: 0.6016 - categorical_accuracy: 0.7099
Epoch 8/200
6/6 [==============================] - 0s 42ms/step - loss: 0.6122 - categorical_accuracy: 0.6975
Epoch 9/200
6/6 [==============================] - 0s 41ms/step - loss: 0.5934 - categorical_accuracy: 0.7099
Epoch 10/2

In [110]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_13 (LSTM)              (None, 40, 128)           130560    
                                                                 
 lstm_14 (LSTM)              (None, 64)                49408     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 2)                 66        
                                                                 
Total params: 186274 (727.63 KB)
Trainable params: 186274 (727.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [111]:
model.save('action.keras')

In [112]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [113]:
yhat = model.predict(X_test)

1/1 [==============================] - 1s 530ms/step


In [114]:
yhat

array([[1.0000000e+00, 7.3750236e-25],
       [9.7942013e-01, 2.0579858e-02],
       [2.4098653e-01, 7.5901347e-01],
       [9.8438984e-01, 1.5610143e-02],
       [1.2827358e-02, 9.8717260e-01],
       [9.9841058e-01, 1.5894111e-03],
       [9.2758495e-01, 7.2415084e-02],
       [9.3459260e-01, 6.5407425e-02],
       [2.1095268e-01, 7.8904730e-01],
       [2.9094009e-08, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [6.8018383e-18, 1.0000000e+00],
       [8.4536463e-01, 1.5463538e-01],
       [9.3742716e-01, 6.2572815e-02],
       [9.8810565e-01, 1.1894394e-02],
       [9.5324409e-01, 4.6755891e-02],
       [9.9472320e-01, 5.2767997e-03],
       [0.0000000e+00, 1.0000000e+00]], dtype=float32)

In [115]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [116]:
ytrue

[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1]

In [117]:
yhat

[0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]

In [118]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 7,  1],
        [ 0, 10]],

       [[10,  0],
        [ 1,  7]]], dtype=int64)

In [119]:
accuracy_score(ytrue, yhat)

0.9444444444444444

In [122]:
sequence = []

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        
        if len(sequence) == 40:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(sign[np.argmax(res)], res)
            del sequence[0]
            
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
cap.release()
cv2.destroyAllWindows()